In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [25]:
Y=np.load('tomatoes.npy') 
print('Length of data: ' + str(len(Y)))
print('Data:')
print(len(Y))
print(np.cov(Y))
print(np.var(Y))

Length of data: 30
Data:
30
5.014596961396833
4.847443729350274


In [43]:
Y3 = np.array([Y,Y,Y])
np.(Y3)

4.847443729350273

In [32]:
def get_minibatch(y,n):
    # x must be an array
    np.random.shuffle(y)
    batch = y[0:n-1]
    y = y
    return batch, y

In [18]:
def SGLD(q,h,force,y,N):
    R = np.random.normal(0,np.sqrt(h),len(q))
    q = q + (h/2) * stochastic_force(q,y,N) + R
    return q

In [6]:
def mSGLD(q,h,force,y,N,batch_size):# n is batch size
    R = np.random.normal(0,1,len(q))
    cov_esti=(N**2/batch_size)*((N-batch_size)/(N-1))*np.cov(Y)
    #cov_U=np.mean((0.5*stochastic_force(q,y,N)-np.mean(0.5*stochastic_force(q,y,N)))*(0.5*stochastic_force(q,y,N)-np.mean(0.5*stochastic_force(q,y,N))))
    q = q + (h/2) * stochastic_force(q,y,N) + np.sqrt(h-h**2*cov_esti/4)*R#(np.identity(3)-0.5*h*cov_U)*R
    return q
    

In [7]:
def stochastic_force(q,y,batch_scaling):
    m1,m2,m3 = q
    # constant number in the likelihood part of all derivatives
    denom = np.exp(-((y-m1)**2)/2)+np.exp(-((y-m2)**2)/2)+np.exp(-((y-m3)**2)/2)
    
    dU_dm1 = (13.5-m1)/16 + batch_scaling*sum((y-m1)*np.exp(-((y-m1)**2)/2)/denom)
    dU_dm2 = (13.5-m2)/16 + batch_scaling*sum((y-m2)*np.exp(-((y-m2)**2)/2)/denom)
    dU_dm3 = (13.5-m3)/16 + batch_scaling*sum((y-m3)*np.exp(-((y-m3)**2)/2)/denom)
    
    dU_dq = np.array([dU_dm1,dU_dm2,dU_dm3])
    return dU_dq

In [11]:
def run_simulation_mSGLD( q0, Nsteps, h, step_function, force_function, y, batch_size):
    
    q_traj = [np.copy(q0)] 
    t_traj = [0]

    q = np.copy(q0) 
    t = 0 
    N = len(y)
    batch_scaling = N/batch_size
    
    for n in range(Nsteps):
        y_batch, y = get_minibatch(y,batch_size)
        q = step_function(q, h, force_function,y_batch,batch_scaling,batch_size)
        t = t + h 
        

        q_traj += [q] 
        t_traj += [t] 

    q_traj = np.array(q_traj) 
    t_traj = np.array(t_traj) 

    return q_traj, t_traj

In [14]:
# generate more tomatoe-like data
N = 1000
d1 = np.random.normal(11.2115,1.5,int(N/3))
d2 = np.random.normal(13.5618,1.5,int((N-int(N/3))/2))
d3 = np.random.normal(16.115,1.5,N-2*int((N-int(N/3))/2))

BigY = np.random.choice(np.concatenate([d1,d2,d3]), N, replace=True)

In [16]:
bat=30

In [17]:
#using mSGLD
q0 = np.array([12,12,12])

Nsteps = 100000
h = 0.005

# Run one long trajectory of Nsteps, using the OBABO scheme
q_mtraj, t_mtraj = run_simulation_mSGLD(q0, Nsteps , h, mSGLD, stochastic_force, BigY, bat)
q_traj, t_traj = run_simulation_SGLD(q0, Nsteps , h, SGLD, stochastic_force, BigY, bat)

In [ ]:
# mSGLD
plt.figure(figsize = (15,6))
plt.subplot(1,2,1)
plt.plot( t_traj, q_traj )
plt.title('Trajectory of $q$')
plt.ylabel('$q(t)$')
plt.xlabel('Time $t$')

histogram_m1,bins_m1 = np.histogram(q_mtraj[:,0],bins=50,range=[10,20], density=True)
midx_m1 = (bins_m1[0:-1]+bins_m1[1:])/2
plt.subplot(1,2,2)

plt.plot(midx_m1,histogram_m1,label='$\mu 1$')

histogram_m2,bins_m2 = np.histogram(q_mtraj[:,1],bins=50,range=[10,20], density=True)
midx_m2 = (bins_m2[0:-1]+bins_m2[1:])/2
plt.plot(midx_m2,histogram_m2,label='$\mu 2$')

histogram_m3,bins_m3 = np.histogram(q_mtraj[:,2],bins=50,range=[10,20], density=True)
midx_m3 = (bins_m3[0:-1]+bins_m3[1:])/2
plt.plot(midx_m3,histogram_m3,label='$\mu 3$')

plt.title('Distribution of $q$')
plt.xlabel('$q$')
plt.ylabel('Density')
plt.legend()
plt.show()

In [ ]:
#mSGLD
new_q_mtraj=np.sort(q_mtraj)

histogram_m1,bins_m1 = np.histogram(new_q_mtraj[:,0],bins=50,range=[7,20], density=True)
midx_m1 = (bins_m1[0:-1]+bins_m1[1:])/2
plt.plot(midx_m1,histogram_m1,label='$\mu 1$')

histogram_m2,bins_m2 = np.histogram(new_q_mtraj[:,1],bins=50,range=[7,20], density=True)
midx_m2 = (bins_m2[0:-1]+bins_m2[1:])/2
plt.plot(midx_m2,histogram_m2,label='$\mu 2$')

histogram_m3,bins_m3 = np.histogram(new_q_mtraj[:,2],bins=50,range=[7,20], density=True)
midx_m3 = (bins_m3[0:-1]+bins_m3[1:])/2
plt.plot(midx_m3,histogram_m3,label='$\mu 3$')

plt.title('Distribution of $q$')
plt.xlabel('$q$')
plt.ylabel('Density')
plt.legend()

print('Mean 1:',np.mean(new_q_mtraj[:,0]), 'Standard Deviation:', np.std(new_q_mtraj[:,0]))
print('Mean 2:',np.mean(new_q_mtraj[:,1]),'Standard Deviation:', np.std(new_q_mtraj[:,1]))
print('Mean 3:',np.mean(new_q_mtraj[:,2]),'Standard Deviation:', np.std(new_q_mtraj[:,2]))